In [2]:
from glob import glob
from itertools import chain
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import requests
from requests.exceptions import JSONDecodeError
import json
import seaborn as sns
import yfinance as yf
from fredapi import Fred
from numba import float64, guvectorize, int64, njit, vectorize
from pandas.tseries.offsets import BMonthEnd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [136]:
#pd.set_option('display.max_columns', None)

In [137]:
def group_by_b_month_end(dt):
    end_date = dt + BMonthEnd(0)
    return end_date

In [12]:
def read_msci_data(filename):
    df = pd.read_excel(filename, skiprows=6, skipfooter=19)
    df.columns = ['date', 'price']
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    df = df.replace(',','', regex=True)
    df['price'] = df['price'].astype(float)
    return df

In [139]:
msci_world = read_msci_data('data/MSCI World USD Gross.xls')

In [140]:
msci_world['drawdown'] = (msci_world['price'] / msci_world['price'].cummax() - 1)

In [141]:
def extract_financialtimes_data(filepaths):
    dfs = []
    for filepath in filepaths:
        dfs.append(pd.read_html(filepath)[2].iloc[::-1])
    df = pd.concat(dfs, ignore_index=True)
    df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: ''.join(x.rsplit(',', maxsplit=2)[-2:])[1:]))
    df = df[df['Date'].isin(pd.date_range(df['Date'].iloc[0], df['Date'].iloc[-1], freq='BM'))]
    df = df.reset_index(drop=True)
    df = df[['Date', 'Close']]
    df.columns = ['date', 'price']
    df = df.set_index('date')
    return df
    

In [142]:
sti = extract_financialtimes_data(glob('data/STI Data/*/*.htm'))

In [143]:
def download_fed_funds_rate():
    fred = Fred()
    fed_funds_rate = fred.get_series('DFF')
    fed_funds_rate.name = 'ffr'
    fed_funds_rate.index.name = 'date'
    fed_funds_rate_1m = fed_funds_rate.div(36000).add(1).groupby(group_by_b_month_end).prod().pow(12).sub(1).mul(100)
    return fed_funds_rate, fed_funds_rate_1m
    

In [144]:
fed_funds_rate, fed_funds_rate_1m = download_fed_funds_rate()

In [145]:
sp500 = yf.download('^SP500TR')['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [146]:
sp500 = sp500.groupby(group_by_b_month_end).last()

In [147]:
def read_shiller_sp500_data(net=False):
    df = pd.read_excel('data/ie_data.xls', 'Data', skiprows=range(7), skipfooter=1).drop(['Unnamed: 13','Unnamed: 15'], axis=1)
    df.index = pd.to_datetime(df['Date'].astype(str).str.split('.').apply(lambda x: '-'.join(x)).str.ljust(7, '0')) + BMonthEnd(0)
    shiller_sp500 = df['P'].add(df['D'].ffill().div(12).mul(0.7 if net else 1)).div(df['P'].shift(1)).fillna(1).cumprod()
    shiller_sp500.name = 'shiller_sp500'
    return shiller_sp500

In [148]:
shiller_sp500 = read_shiller_sp500_data()

In [150]:
def download_usdsgd():
    usd_sgd_response = requests.get('https://eservices.mas.gov.sg/api/action/datastore/search.json',
                   params={'resource_id': '10eafb90-11a2-4fbd-b7a7-ac15a42d60b6',
                           'between[end_of_month]': f'1969-12,{pd.to_datetime("today").strftime("%Y-%m")}',
                           'fields': 'end_of_month,usd_sgd'
                           }
                   ).json()
    usdsgd = pd.DataFrame(usd_sgd_response['result']['records'])[['end_of_month', 'usd_sgd']]
    usdsgd['end_of_month'] = pd.to_datetime(usdsgd['end_of_month']) + BMonthEnd()
    return usdsgd

In [151]:
usdsgd = download_usdsgd()

In [152]:
def download_sgd_interest_rates():
    offset = 0
    dfs = []
    while True:
        sgd_interest_rates_response = requests.get('https://eservices.mas.gov.sg/api/action/datastore/search.json',
                    params={'resource_id': '9a0bf149-308c-4bd2-832d-76c8e6cb47ed',
                            'between[end_of_day]': f'1987-07-01,{pd.to_datetime("today").strftime("%Y-%m-%d")}',
                            'offset': f'{offset}',
                            'fields': 'end_of_day,interbank_overnight,sora'
                            }
                    ).json()
        df = pd.DataFrame(sgd_interest_rates_response['result']['records'])[['end_of_day', 'interbank_overnight', 'sora']]
        offset += 100
        dfs.append(df)
        if len(df) < 100:
            break
    sgd_interest_rates = pd.concat(dfs)
    sgd_interest_rates['interbank_overnight'] = sgd_interest_rates['interbank_overnight'].astype(float)
    sgd_interest_rates['end_of_day'] = pd.to_datetime(sgd_interest_rates['end_of_day'])
    sgd_interest_rates = sgd_interest_rates.dropna(how='all', subset=['interbank_overnight', 'sora'])
    sgd_interest_rates = sgd_interest_rates.drop_duplicates().drop_duplicates(subset=['end_of_day', 'interbank_overnight']).drop_duplicates(subset=['end_of_day', 'sora'])
    sgd_interest_rates = sgd_interest_rates.reset_index(drop=True)
    sgd_interest_rates = sgd_interest_rates.set_index('end_of_day')
    return sgd_interest_rates

In [153]:
def load_sgd_interest_rates():
    try:
        sgd_interest_rates = pd.read_csv('data/sgd_interest_rates.csv', parse_dates=['end_of_day'])
        if pd.to_datetime(sgd_interest_rates['end_of_day']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1):
            raise FileNotFoundError
        sgd_interest_rates = sgd_interest_rates.set_index('end_of_day')
        return sgd_interest_rates
    except FileNotFoundError:
        sgd_interest_rates = download_sgd_interest_rates()
        sgd_interest_rates.to_csv('data/sgd_interest_rates.csv')
        return sgd_interest_rates

In [154]:
sgd_interest_rates = load_sgd_interest_rates()

In [155]:
sgd_interest_rates_1m = sgd_interest_rates.resample('D').ffill().div(36000).add(1).groupby(group_by_b_month_end).prod().pow(12).sub(1).mul(100).replace(0, np.nan)

In [156]:
sgd_interest_rates_1m.loc['2014-01-31', 'interbank_overnight'] = np.nan

In [157]:
sgd_interest_rates_1m['sgd_ir_1m'] = sgd_interest_rates_1m['interbank_overnight'].fillna(sgd_interest_rates['sora'])

In [158]:
def download_sg_cpi():
    try:
        sg_cpi_response = requests.get('https://tablebuilder.singstat.gov.sg/api/table/tabledata/M212882')
        sg_cpi = pd.DataFrame(sg_cpi_response.json()['Data']['row'][0]['columns'])
        sg_cpi.columns = ['date', 'sg_cpi']
        sg_cpi['date'] = pd.to_datetime(sg_cpi['date']) + BMonthEnd()
        sg_cpi = sg_cpi.set_index('date')
    except JSONDecodeError:
        sg_cpi = pd.read_csv('data/sg_cpi.csv', index_col='date')
    return sg_cpi

In [159]:
def load_sg_cpi():
    try:
        sg_cpi = pd.read_csv('data/sg_cpi.csv', parse_dates=['date'])
        if pd.to_datetime(sg_cpi['date']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1, 'D'):
            raise FileNotFoundError
        sg_cpi = sg_cpi.set_index('date')
        return sg_cpi
    except FileNotFoundError:
        sg_cpi = download_sg_cpi()
        sg_cpi.to_csv('data/sg_cpi.csv')
        return sg_cpi

In [160]:
sg_cpi = load_sg_cpi()

In [161]:
def download_us_cpi():
    with requests.Session() as session:
        dfs = []
        for i in range(1947, 2023, 10):
            us_cpi_response = session.post(
            'https://api.bls.gov/publicAPI/v2/timeseries/data/',
            json={'seriesid': ['CUSR0000SA0'],
                'startyear': f'{i}',
                'endyear': f'{i+9}',
                'catalog': 'true',
                'registrationkey': os.environ['BLS_API_KEY']
                },
            headers={'Content-Type': 'application/json'}
            )
            dfs.append(pd.DataFrame(us_cpi_response.json()['Results']['series'][0]['data']).iloc[::-1])
        us_cpi = pd.concat(dfs).reset_index(drop=True)
    us_cpi['month'] = us_cpi['period'].str[-2:]
    us_cpi['date'] = pd.to_datetime(us_cpi['year'] + '-' +us_cpi['month']) + BMonthEnd()
    us_cpi['value'] = us_cpi['value'].astype(float)
    us_cpi = us_cpi[['date', 'value']]
    us_cpi.columns = ['date', 'us_cpi']
    us_cpi = us_cpi.set_index('date')
    return us_cpi

In [162]:
def load_us_cpi():
    try:
        us_cpi = pd.read_csv('data/us_cpi.csv', parse_dates=['date'])
        if pd.to_datetime(us_cpi['date']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1, 'D'):
            raise FileNotFoundError
        us_cpi = us_cpi.set_index('date')
        return us_cpi
    except FileNotFoundError:
        us_cpi = download_us_cpi()
        us_cpi.to_csv('data/us_cpi.csv')
        return us_cpi

In [163]:
us_cpi = load_us_cpi()

In [164]:
msci_world = msci_world.merge(fed_funds_rate_1m, left_index=True, right_index=True, how='left')

In [165]:
msci_world = msci_world.merge(sgd_interest_rates_1m['sgd_ir_1m'], left_index=True, right_index=True, how='left')

In [166]:
periods = ['1m', '3m', '6m', '1y', '2y', '3y', '5y', '10y', '15y', '20y', '25y', '30y']
durations = [1, 3, 6, 12, 24, 36, 60, 120, 180, 240, 300, 360]

In [167]:
@njit
def calculate_return(ending_index, dca_length, monthly_returns, investment_horizon=None):
    if investment_horizon is None:
        investment_horizon = dca_length
    elif investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    if ending_index < dca_length:
        return np.nan
    share_value = 0
    cash = 1
    for i in range(ending_index - investment_horizon, ending_index - investment_horizon + dca_length):
        cash -= 1/dca_length
        share_value += 1/dca_length
        share_value *= 1 + monthly_returns[i+1]
    for i in range(ending_index - investment_horizon + dca_length, ending_index):
        share_value *= 1 + monthly_returns[i+1]
    return share_value - 1

@guvectorize([(int64, float64[:], int64, float64[:])], '(),(n),()->(n)', target='parallel', nopython=True)
def calculate_return_vector(dca_length, monthly_returns, investment_horizon, res=np.array([])):
    if investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    for i in range(len(monthly_returns)):
        if i < investment_horizon:
            res[i] = np.nan
        share_value = 0
        cash = 1
        for j in range(i - investment_horizon, i - investment_horizon + dca_length):
            cash -= 1/dca_length
            share_value += 1/dca_length
            share_value *= 1 + monthly_returns[j+1]
        for j in range(i - investment_horizon + dca_length, i):
            share_value *= 1 + monthly_returns[j+1]
        res[i] = share_value - 1

@guvectorize([(float64, float64, float64, float64, int64, int64, int64, float64[:], float64[:], float64[:])], '(),(),(),(),(),(),(),(n),(n)->(n)', target='parallel', nopython=True)
def calculate_lumpsum_return_with_fees_and_interest_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, total_investment, dca_length, dca_interval, investment_horizon, monthly_returns, interest_rates, res=np.array([])):
    if investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    if fixed_transaction_fees >= total_investment / dca_length * dca_interval:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < investment_horizon:
            res[i] = np.nan
        share_value = 0
        cash = total_investment
        monthly_amount = total_investment / dca_length
        for index, j in enumerate(range(i - investment_horizon, i - investment_horizon + dca_length)):
            if index % dca_interval == 0:
                dca_amount = cash - (dca_length - index - 1) * monthly_amount
                share_value += dca_amount * (1 - variable_transaction_fees) - fixed_transaction_fees
                cash = (dca_length - index - 1) * monthly_amount
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
            cash *= (1 + interest_rates[j+1] / 100) ** (1/12)
        share_value += cash
        cash = 0
        for j in range(i - investment_horizon + dca_length, i):
            share_value *= 1 + monthly_returns[j+1]
        res[i] = (share_value - total_investment) / total_investment

@guvectorize([(float64, float64, float64, float64, int64, int64, float64[:], float64[:], float64[:])], '(),(),(),(),(),(),(n),(n)->(n)', target='parallel', nopython=True)
def calculate_dca_return_with_fees_and_interest_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, monthly_amount, dca_length, dca_interval, monthly_returns, interest_rates, res=np.array([])):
    total_investment = monthly_amount * dca_length
    dca_amount = monthly_amount * dca_interval
    if fixed_transaction_fees >= dca_amount:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < dca_length:
            res[i] = np.nan
        share_value = 0
        funds_to_invest = 0
        for index, j in enumerate(range(i - dca_length, i)):
            funds_to_invest += monthly_amount
            if (index + 1) % dca_interval == 0:
                share_value += funds_to_invest * (1 - variable_transaction_fees) - fixed_transaction_fees
                funds_to_invest = 0
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
            funds_to_invest *= (1 + interest_rates[j+1] / 100) ** (1/12)
        res[i] = (share_value + funds_to_invest - total_investment) / total_investment

@guvectorize([(float64, float64, float64, float64, float64, int64, float64[:], float64[:], float64[:], float64[:])], '(),(),(),(),(),(),(n),(n),(n)->(n)', target='parallel', nopython=True)
def calculate_dca_buythedip_return_with_fees_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, monthly_investment, monthly_savings, dca_length, monthly_returns, fed_funds_rate, drawdown, res=np.array([])):
    total_investment = (monthly_investment + monthly_savings) * dca_length
    if fixed_transaction_fees >= monthly_investment:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < dca_length:
            res[i] = np.nan
        share_value = 0
        amount_invested = 0
        warchest = 0
        for j in range(i - dca_length, i):
            share_value += monthly_investment * (1 - variable_transaction_fees) - fixed_transaction_fees
            amount_invested += monthly_investment
            warchest += monthly_savings
            if drawdown[j] < -0.20:
                share_value += warchest * (1 - variable_transaction_fees) - fixed_transaction_fees
                amount_invested += warchest
                warchest = 0
            warchest *= (1 + fed_funds_rate[j+1] / 100) ** (1/12)
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
        res[i] = (share_value - amount_invested) / total_investment

In [168]:
def add_return_columns(df):
    for period, duration in zip(periods, durations):
        df[f'{period}_cumulative'] = df['price'].pct_change(periods=duration)
    for period, duration in zip(periods, durations):
        df[f'{period}_annualized'] = (1 + df[f'{period}_cumulative'])**(12/duration) - 1
    for period, duration in zip(periods, durations):
        df[f'{period}_dca_cumulative'] = calculate_return_vector(duration, df['1m_cumulative'].values, duration)
    for period, duration in zip(periods, durations):
        df[f'{period}_dca_annualized'] = (1 + df[f'{period}_dca_cumulative'])**(12/duration) - 1
    for period, duration in zip(periods, durations):
        df[f'{period}_cumulative_difference'] = df[f'{period}_cumulative'] - df[f'{period}_dca_cumulative']
    for period, duration in zip(periods, durations):
        df[f'{period}_difference_in_annualized'] = df[f'{period}_annualized'] - df[f'{period}_dca_annualized']

In [169]:
add_return_columns(msci_world)

In [170]:
add_return_columns(sti)

In [171]:
msci_world.head(10)

,price,drawdown,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,...,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,15y_difference_in_annualized,20y_difference_in_annualized,25y_difference_in_annualized,30y_difference_in_annualized
date,,,,,,,,,,,,,,,,,,,,,
1969-12-31,100.000,0.00000,10.391636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-30,94.455,-0.05545,9.376079,NaN,-0.055450,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-27,97.405,-0.02595,8.771530,NaN,0.031232,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,97.708,-0.02292,8.657911,NaN,0.003111,-0.022920,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,88.578,-0.11422,8.432125,NaN,-0.093442,-0.062220,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-29,82.990,-0.17010,7.988168,NaN,-0.063086,-0.147990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-06-30,80.946,-0.19054,8.469048,NaN,-0.024629,-0.171552,-0.190540,NaN,NaN,NaN,...,-0.100260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-07-31,85.965,-0.14035,7.739472,NaN,0.062004,-0.029499,-0.089884,NaN,NaN,NaN,...,-0.087167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-08-31,88.799,-0.11201,7.074040,NaN,0.032967,0.069996,-0.088353,NaN,NaN,NaN,...,-0.176561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
msci_world.describe()

,price,drawdown,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,...,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,15y_difference_in_annualized,20y_difference_in_annualized,25y_difference_in_annualized,30y_difference_in_annualized
count,644.000000,644.000000,644.000000,430.000000,643.000000,641.000000,638.000000,632.000000,620.000000,608.000000,...,638.000000,632.000000,620.000000,608.000000,584.000000,524.000000,464.000000,404.000000,344.000000,284.000000
mean,2210.263011,-0.086298,5.193560,1.713778,0.008044,0.024482,0.050332,0.103953,0.220356,0.343222,...,0.051814,0.048165,0.046528,0.044904,0.042556,0.041678,0.039932,0.040511,0.041230,0.041236
std,2299.939234,0.110092,4.284294,1.661664,0.043077,0.077157,0.115190,0.171387,0.271823,0.363288,...,0.125752,0.095777,0.071055,0.057231,0.042654,0.025945,0.020795,0.016191,0.011568,0.008780
min,80.946000,-0.540291,0.048345,0.017001,-0.189601,-0.332204,-0.435526,-0.471197,-0.474002,-0.456299,...,-0.326231,-0.341797,-0.170335,-0.118299,-0.063706,-0.018452,-0.005248,0.002925,0.017413,0.025894
25%,260.754750,-0.134239,1.449395,0.305900,-0.016660,-0.012811,-0.014083,0.007289,0.077655,0.158346,...,-0.021686,-0.004104,0.008902,0.017811,0.020580,0.024311,0.025343,0.029690,0.031768,0.033042
50%,1528.714000,-0.043185,5.140584,1.255930,0.011805,0.027715,0.054160,0.121249,0.240952,0.321857,...,0.046812,0.050672,0.052689,0.050703,0.044409,0.043291,0.037928,0.037653,0.037921,0.041795
75%,3204.294250,0.000000,7.405355,2.755771,0.032711,0.070665,0.112827,0.200170,0.363238,0.519561,...,0.117783,0.100591,0.085937,0.078488,0.065705,0.057511,0.055861,0.055766,0.052008,0.049330
max,9755.694000,0.000000,23.069445,8.139264,0.145823,0.304416,0.466621,0.659709,1.372536,1.969380,...,0.649668,0.392638,0.288147,0.203891,0.161209,0.106434,0.088574,0.074823,0.066975,0.059140


In [173]:
msci_world.loc[:, [*msci_world.loc[:,'1m_annualized':'30y_annualized'].columns, *msci_world.loc[:,'1m_dca_annualized':'30y_dca_annualized']]].describe()

,1m_annualized,3m_annualized,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,15y_annualized,20y_annualized,...,6m_dca_annualized,1y_dca_annualized,2y_dca_annualized,3y_dca_annualized,5y_dca_annualized,10y_dca_annualized,15y_dca_annualized,20y_dca_annualized,25y_dca_annualized,30y_dca_annualized
count,643.000000,641.000000,638.000000,632.000000,620.000000,608.000000,584.000000,524.000000,464.000000,404.000000,...,638.000000,632.000000,620.000000,608.000000,584.000000,524.000000,464.000000,404.000000,344.000000,284.000000
mean,0.230886,0.138309,0.116446,0.103953,0.097792,0.094597,0.092672,0.094241,0.094007,0.092712,...,0.064631,0.055789,0.051264,0.049694,0.050115,0.052563,0.054075,0.052202,0.051709,0.051014
std,0.613277,0.329712,0.239930,0.171387,0.123426,0.098293,0.074162,0.046920,0.040047,0.031516,...,0.146126,0.100533,0.071832,0.057880,0.044269,0.029104,0.025956,0.021609,0.018721,0.012944
min,-0.919761,-0.801127,-0.681369,-0.471197,-0.274743,-0.183818,-0.056829,-0.025486,0.028006,0.032193,...,-0.512605,-0.356104,-0.245937,-0.170785,-0.088353,-0.031450,-0.008844,0.006518,0.020997,0.032972
25%,-0.182585,-0.050269,-0.027968,0.007289,0.038102,0.050218,0.039414,0.064961,0.060429,0.065297,...,-0.010019,0.003418,0.017137,0.020197,0.023542,0.033674,0.035411,0.034995,0.035808,0.040892
50%,0.151232,0.115555,0.111254,0.121249,0.113980,0.097475,0.093844,0.088535,0.084853,0.090881,...,0.067259,0.066888,0.062834,0.056712,0.049532,0.049496,0.047851,0.044507,0.045958,0.044972
75%,0.471445,0.314060,0.238384,0.200170,0.167578,0.149669,0.132127,0.126265,0.136756,0.120894,...,0.149061,0.112486,0.090978,0.081676,0.072289,0.066945,0.076667,0.077222,0.064854,0.063469
max,4.121682,1.895109,1.150978,0.659709,0.540304,0.437326,0.336423,0.200539,0.174726,0.154128,...,0.575100,0.372030,0.289270,0.256893,0.198666,0.136200,0.107792,0.090732,0.091150,0.085453


In [174]:
go.Figure(
    data = [
        go.Box(
            x=msci_world[column],
            name=column,
            )
        for column in msci_world.loc[:,'1m_annualized':'30y_annualized'].columns
    ],
    layout = go.Layout(
        height=800,
        xaxis=dict(
            tickformat='.2%',
        )
    )
)

In [175]:
go.Figure(
    data = [
        go.Box(
            x=msci_world[column],
            name=column,
            )
        for column in chain.from_iterable(zip(msci_world.loc[:,'1m_annualized':'30y_annualized'].columns, msci_world.loc[:,'1m_dca_annualized':'30y_dca_annualized']))
    ],
    layout = go.Layout(
        height=800,
        xaxis=dict(
            tickformat='.2%',
        )
    )
)

In [176]:
go.Figure(
    [
        go.Scatter(
            x=msci_world.index,
            y=msci_world[column],
            name=column,
            mode='lines'
            )
        for column in ['5y_annualized', '5y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [177]:
go.Figure(
    [
        go.Scatter(
            x=msci_world.index,
            y=msci_world[column],
            name=column,
            mode='lines'
            )
        for column in ['10y_annualized', '10y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [178]:
go.Figure(
    [
        go.Scatter(
            x=msci_world.index,
            y=msci_world[column],
            name=column,
            mode='lines'
            )
        for column in ['20y_annualized', '20y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [179]:
go.Figure(
    [
        go.Box(
            x=msci_world[column],
            name=column,
            opacity=0.75
            )
        for column in msci_world.loc[:, '1m_difference_in_annualized':'30y_difference_in_annualized'].columns
    ],
    layout = go.Layout(
        xaxis=dict(
            tickformat='.0%',
        )
    )
)